#TASK 2: Restaurant Recommendation
Cognifyz Technologies

Shivani Goyal

## Importing Necessary Libraries

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob

# Download NLTK stopwords and punkt tokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
rd=pd.read_csv("/content/drive/MyDrive/Dataset .csv")

In [ ]:
rd.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [ ]:
rd.describe()

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Price range,Aggregate rating,Votes
count,9.551000e+03,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000
mean,9.051128e+06,18.365616,64.126574,25.854381,1199.210763,1.804837,2.666370,156.909748
std,8.791521e+06,56.750546,41.467058,11.007935,16121.183073,0.905609,1.516378,430.169145
min,5.300000e+01,1.000000,-157.948486,-41.330428,0.000000,1.000000,0.000000,0.000000
25%,3.019625e+05,1.000000,77.081343,28.478713,250.000000,1.000000,2.500000,5.000000
50%,6.004089e+06,1.000000,77.191964,28.570469,400.000000,2.000000,3.200000,31.000000
75%,1.835229e+07,1.000000,77.282006,28.642758,700.000000,2.000000,3.700000,131.000000
max,1.850065e+07,216.000000,174.832089,55.976980,800000.000000,4.000000,4.900000,10934.000000


In [ ]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [ ]:
# Removing the Duplicates
rd.drop_duplicates(inplace=True)

In [ ]:
# Deleting Unnecessary Columns
columns_to_drop = ['Country Code', 'Address', 'Locality', 'Locality Verbose', 'Currency', 'Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu', 'Rating color', 'Rating text']
rd.drop(columns=columns_to_drop, inplace=True)

In [ ]:
rd.isna().sum()

Restaurant ID           0
Restaurant Name         0
City                    0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Price range             0
Aggregate rating        0
Votes                   0
dtype: int64

In [ ]:
#Handlling missing values in 'Cuisines'
rd.fillna('Cuisines', inplace=True)  # For categorical columns

In [ ]:
# Changing the column names
new_column_names = {
    'Restaurant ID': 'restaurant_id',
    'Restaurant Name': 'restaurant_name',
    'City': 'city',
    'Longitude': 'longitude',
    'Latitude': 'latitude',
    'Cuisines': 'cuisines',
    'Average Cost for two': 'average_cost_for_two',
    'Price range': 'price_range',
    'Aggregate rating': 'aggregate_rating',
    'Votes': 'votes'
}
rd.rename(columns=new_column_names, inplace=True)

# Adjust the column names
rd.columns = rd.columns.str.lower().str.replace(' ', '_')

## Generate TF-IDF Matrix and Calculate Cosine Similarity

In [ ]:
# TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(rd['cuisines'])

# cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


##Defining the Recommendation Function

In [ ]:
# Function to get recommendations based on cosine similarity and user input
def get_recommendations(cuisine_preference, price_range, cosine_sim=cosine_sim):
    # Filter the dataset based on user inputs
    filtered_df = rd[(rd['cuisines'].str.contains(cuisine_preference, case=False)) &
                     (rd['price_range'] == price_range)]

    if filtered_df.empty:
        return "No restaurants found matching the criteria. Please try different preferences."

    # Recompute the TF-IDF matrix for the filtered dataset
    tfidf_filtered_matrix = tfidf.fit_transform(filtered_df['cuisines'])

    # Calculate cosine similarity for the filtered dataset
    cosine_sim_filtered = linear_kernel(tfidf_filtered_matrix, tfidf_filtered_matrix)

    # Get the index of the first restaurant in the filtered dataset (as a reference)
    idx = 0

    # Get the pairwise similarity scores of all restaurants with the reference restaurant
    sim_scores = list(enumerate(cosine_sim_filtered[idx]))

    # Sort the restaurants based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar restaurants
    sim_scores = sim_scores[1:11]

    # Get the restaurant indices
    restaurant_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar restaurants
    return filtered_df.iloc[restaurant_indices][['restaurant_name', 'cuisines', 'city', 'average_cost_for_two', 'aggregate_rating']]



##Taking User input

In [ ]:
# Test the recommendation system with an example
cuisine_preference = input("Enter your preferred cuisine: ")
price_range = int(input("Enter your budget (1-4): "))  # Assuming price range is categorized from 1 to 4

recommendations = get_recommendations(cuisine_preference, price_range)
print(recommendations)

Enter your preferred cuisine: Sushi
Enter your budget (1-4): 3
                             restaurant_name                cuisines  \
98               Shogun Japanese Steak House  Japanese, Steak, Sushi   
195     Samurai Japanese Cuisine & Sushi Bar  Japanese, Steak, Sushi   
279   Ichiban Hibachi Steakhouse & Sushi Bar  Japanese, Steak, Sushi   
342    Zen Japanese Steakhouse and Sushi Bar  Japanese, Steak, Sushi   
491    Tokyo Japanese Steakhouse & Sushi Bar  Japanese, Steak, Sushi   
215                 Fuji Japanese Steakhouse         Japanese, Sushi   
247                                    Osaka         Japanese, Sushi   
480             Fuji Bay Japanese Restaurant         Japanese, Sushi   
549                                   Sakura         Japanese, Sushi   
1319                              Sushi Haus         Japanese, Sushi   

            city  average_cost_for_two  aggregate_rating  
98        Albany                    40               3.5  
195     Columbus          